###  Using the class spectral analysis

Tutorial on how to use the spectral analysis class to compute power spectrum, spectogram, and the pairwise coherence.

In [1]:
%matplotlib inline
import time
import numpy                 as     np
import matplotlib.pyplot     as     plt
from   GDa.spectral_analysis import spectral_analysis
from   GDa.session           import session
from   GDa.misc              import smooth_spectra

Let's start by loading a session data:

In [ ]:
# Session data
session_data = np.load('raw_lfp/lucy_session01_150128.npy', allow_pickle=True).item()
# LFP data
LFP          = session_data['data']
# Index of all pair of channels
pairs        = session_data['info']['pairs']
# Sample frequency
fsample      = int(session_data['info']['fsample'])
# Directory were to save the coherence data
dir_out      = session_data['path']['dir_out']

To instantiate the __spectral_analysis__ class:

In [ ]:
spec = spectral_analysis()

We can plot a random LFP signal:

In [ ]:
lfp   = LFP[0,10,:]
# Time axis, using from -0.65 to 3.00 s because this is the event dt we used when we saved the data
tarr = np.linspace(-0.65, 3.00, lfp.shape[-1]) 

In [ ]:
# Plotting the signal
plt.plot(tarr, lfp, 'k')
plt.title('Selected LFP')
plt.xlabel('Time')
plt.ylabel('mV')

We can bandpass filter the data with the function __filter__ in __spectral_analysis__, let's filter it on the alpha band (8-15 Hz):

In [ ]:
LFP_filtered = spec.filter(data = LFP, fs = fsample, f_low = 8, f_high = 12, n_jobs = -1)

In [ ]:
plt.plot(lfp, 'k')
plt.plot(LFP_filtered[0,10,:], 'r')
plt.title('Filtering the LFP')
plt.legend(['Orignal', '8-15 Hz'])

We can compute the wavelets of the signal using the function __wavelet_transform__, which allows to use morlet or multitaper transforms.

In [ ]:
# Defining frequencies
freqs = np.arange(4,60,2)

# Morlet
W_ml = spec.wavelet_transform(data = LFP,  
                              fs = fsample, freqs = freqs, n_cycles = 5,
                              time_bandwidth = None, delta = 15, method = 'morlet', n_jobs = -1)

# Multitaper
#W_mt = spec.wavelet_transform(data = LFP,  
#                              fs = fsample, freqs = freqs, n_cycles = freqs/2.0,
#                              time_bandwidth = 8.0, delta = 15, method = 'multitaper', n_jobs = -1)

The auto spectra can be computed as tem elementwise product of $W$ and its conjugate:

In [ ]:
Sauto_ml = W_ml * np.conj(W_ml)
#Sauto_mt = W_mt * np.conj(W_mt)

Next, we can visualize the result for __trial=0__, and __channel=10__:

In [ ]:
#plt.figure(figsize=(10,4))
# Sxx
#plt.subplot(1,2,1)
plt.imshow(Sauto_ml[0,10,:,:].real, aspect='auto', cmap='jet', origin='lower', extent=[-0.65,3.00,4,60]);plt.colorbar()
plt.title('Morlet Transform', fontsize=20)
plt.ylabel(r"$\bf{S_{xx}}$" + "\n" + "Frequency [Hz]", fontsize=20)
plt.xlabel('Time [s]')
#plt.subplot(1,2,2)
#plt.imshow(Sauto_mt[0,10,:,:].real, aspect='auto', cmap='jet', origin='lower', extent=[-0.65,3.00,3,40]);plt.colorbar()
#plt.title('Multitaper Transform', fontsize=20)
#plt.xlabel('Time [s]')
plt.tight_layout()

Finally we can compute the (single trial) time-frequency coherence between two signals by using the function __wavelet_coherence__, for each trial and pair a file will be saved in __dir_out__ with the coherence data.

An array with the pairs of channels (pairs = $[[0,1], [0,2], \dots, [n,n]]$) should be passed to the function (note that you don't necessarily have to provide every pair possible, you can also focus on a few pairs you are interested).

If $n_{\rm jobs}>1$, it will parallelize on pairs of channels, to test let's compute it passing only one trial of the data, and measure the execution time:

In [ ]:
start = time.time()

spec.wavelet_coherence(data = LFP[:10,:,:], pairs = pairs, fs = fsample, freqs = freqs, 
                       n_cycles = 5, time_bandwidth = None, delta = 15, method = 'morlet', 
                       win_time = 34, win_freq = 1, dir_out = dir_out, n_jobs = -1)

end = time.time()

print('Elapsed time to load super-tensor: ' +str((end - start)) + ' sec.' )

We can load a saved file with the coherence in order to plot:

In [ ]:
import h5py
hf  = h5py.File('Results/lucy/150128/session01/ch1_10_ch2_30.h5', 'r')
coh = np.array(hf.get('coherence'))

In [ ]:
plt.imshow(coh[0,:,:], aspect = 'auto', cmap = 'jet', origin = 'lower', extent=[-0.65,3,4,60]); plt.colorbar()
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')

The spectral analysis class also have the __gabor_coherence__ method which allows computing the coherence between a pair os signals, since it is not built uppon MNE the parameters we should pass are not the data but two single LFP signals, let's call this funtion for the same signals we have plot above (__trial=0__, __channel1=10__, and __channel2=30__).

In [ ]:
signal1 = LFP[0,10,:]
signal2 = LFP[0,30,:]

coh_gabor = spec.gabor_coherence(signal1 = signal1, signal2 = signal2, fs = fsample, freqs = freqs, 
                                 win_time = 500, win_freq = 1, n_cycles = 5)

In [ ]:
plt.imshow(coh_gabor.real, aspect = 'auto', cmap = 'jet', origin = 'lower', extent=[-0.65,3,4,60]); plt.colorbar()
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')

As it can be seen the results as pretty similar, as expected.